In [1]:
%pip install ../..

Processing /home/hyang/SpuCo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=80928 sha256=22d91c6ed0249cc0beb978eec9393bdd68d40e3dbb5d1b98fd60ad90b7c605db
  Stored in directory: /tmp/pip-ephem-wheel-cache-211j3ox4/wheels/16/2e/00/5bdefcfd7f850d6f19880ecdb4dd3f325f4b906bf004cd82e3
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:3")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_HARD

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

/home/hyang/SpuCo/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.invariant_train import InvariantTrainsetWrapper
invariant_trainset = InvariantTrainsetWrapper(trainset, trainset.group_partition)

ImportError: cannot import name 'InvariantTrainsetWrapper' from 'spuco.invariant_train' (/home/hyang/SpuCo/.conda/lib/python3.10/site-packages/spuco/invariant_train/__init__.py)

In [ ]:
from torch.optim import SGD
from spuco.models import MLP
from spuco.invariant_train import GroupDRO

model = MLP(3*28*28, len(classes)).to(device)
group_dro = GroupDRO(
    model=model,
    num_epochs=5,
    trainset=invariant_trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [ ]:
group_dro.train()

Epoch 4: 100%|██████████| 938/938 [00:22<00:00, 41.73batch/s, accuracy=93.75%, loss=0.0813]  


In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 99.05437352245863
Group (0, 1) Test Accuracy: 91.725768321513
Group (0, 2) Test Accuracy: 93.14420803782505
Group (0, 3) Test Accuracy: 88.4160756501182
Group (0, 4) Test Accuracy: 93.14420803782505
Group (1, 0) Test Accuracy: 94.62102689486552
Group (1, 1) Test Accuracy: 97.31051344743277
Group (1, 2) Test Accuracy: 89.70588235294117
Group (1, 3) Test Accuracy: 65.19607843137256
Group (1, 4) Test Accuracy: 72.05882352941177
Group (2, 0) Test Accuracy: 64.0
Group (2, 1) Test Accuracy: 88.53333333333333
Group (2, 2) Test Accuracy: 96.53333333333333
Group (2, 3) Test Accuracy: 88.53333333333333
Group (2, 4) Test Accuracy: 54.54545454545455
Group (3, 0) Test Accuracy: 67.08542713567839
Group (3, 1) Test Accuracy: 62.97229219143577
Group (3, 2) Test Accuracy: 86.39798488664988
Group (3, 3) Test Accuracy: 96.97732997481108
Group (3, 4) Test Accuracy: 89.67254408060454
Group (4, 0) Test Accuracy: 88.41309823677582
Group (4, 1) Test Accuracy: 66.49874055415617
Grou

{(0, 0): 99.05437352245863,
 (0, 1): 91.725768321513,
 (0, 2): 93.14420803782505,
 (0, 3): 88.4160756501182,
 (0, 4): 93.14420803782505,
 (1, 0): 94.62102689486552,
 (1, 1): 97.31051344743277,
 (1, 2): 89.70588235294117,
 (1, 3): 65.19607843137256,
 (1, 4): 72.05882352941177,
 (2, 0): 64.0,
 (2, 1): 88.53333333333333,
 (2, 2): 96.53333333333333,
 (2, 3): 88.53333333333333,
 (2, 4): 54.54545454545455,
 (3, 0): 67.08542713567839,
 (3, 1): 62.97229219143577,
 (3, 2): 86.39798488664988,
 (3, 3): 96.97732997481108,
 (3, 4): 89.67254408060454,
 (4, 0): 88.41309823677582,
 (4, 1): 66.49874055415617,
 (4, 2): 63.727959697733,
 (4, 3): 89.39393939393939,
 (4, 4): 98.23232323232324}

In [ ]:
evaluator.worst_group_accuracy

((2, 4), 54.54545454545455)

In [ ]:
evaluator.average_accuracy

97.55885932606213

In [ ]:
evaluator.evaluate_spurious_task()

28.44

In [ ]:
import subprocess
import sys
import numpy as np
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
install("../..")
import os
import torch 
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T
from spuco.datasets import GroupLabeledDatasetWrapper
from torch.optim import SGD
from spuco.invariant_train import GroupDRO
from spuco.evaluate import Evaluator
from spuco.models import model_factory 
from spuco.utils import set_seed


device = torch.device("cuda:2")
# lrs = [1e-3]
momentum = 0.9
num_epochs = 50
batch_size = 32
combo=[(0.001,0.0001), (0.001,0.0005), (0.001,0.001),(0.001,0.005)]
# weight_decays=[1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2]
feature_noise = 0
label_noise = 0
magnitude = [SpuriousFeatureDifficulty.MAGNITUDE_HARD]  
# for lr in lrs:
#     for weight_decay in weight_decays:
for i in combo:
        for difficulty in magnitude:

            for seed in [1,2]:
                lr = i[0]
                weight_decay=i[1]
                name = "eval/groupdro_lr_{}_wd_{}_difficulty_{}_seed_{}".format(lr, weight_decay, difficulty, seed)
                if os.path.exists(name + '.npz'):
                    continue
                else:
                    set_seed(seed)
                    classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
                    trainset = SpuCoMNIST(
                        root="/data/mnist/",
                        spurious_feature_difficulty=difficulty,
                        core_feature_noise=feature_noise,
                        label_noise = label_noise,
                        spurious_correlation_strength=0.995,
                        classes=classes,
                        split="train"
                    )
                    trainset.initialize()

                    testset = SpuCoMNIST(
                        root="/data/mnist/",
                        spurious_feature_difficulty=difficulty,
                        classes=classes,
                        split="test"
                    )
                    testset.initialize()

                    valset = SpuCoMNIST(
                        root="/data/mnist/",
                        spurious_feature_difficulty=difficulty,
                        classes=classes,
                        split="val"
                    )
                    valset.initialize()

                    group_trainset = GroupLabeledDatasetWrapper(trainset, trainset.group_partition)

                    model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
                    group_dro = GroupDRO(
                        model=model,
                        num_epochs=num_epochs,
                        trainset=group_trainset,
                        valset=valset,
                        batch_size=batch_size,
                        optimizer=SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=True, weight_decay=weight_decay),
                        device=device,
                        verbose=True
                    )
                    group_dro.train()
                    model = group_dro.best_model

                    evaluator = Evaluator(
                        testset=testset,
                        group_partition=testset.group_partition,
                        group_weights=trainset.group_weights,
                        batch_size=64,
                        model=model,
                        device=device,
                        verbose=True
                    )
                    evaluator.evaluate()

                    np.savez("eval/groupdro_lr_{}_wd_{}_difficulty_{}_seed_{}".format(lr, weight_decay, difficulty, seed), worst_acc=evaluator.worst_group_accuracy[1], ave_acc=evaluator.average_accuracy)